# Momentos de HU - Base Four Shape


## Objetivo Geral:
Utilizando os códigos Matlab/Octave juntamente com a base de formas, mostre a acurácia para a tarefa. Utilize 60% para treinamento e 40% para teste. Selecione as 600 primeiras imagens para treino e a restante para teste.


## Objetivos Específicos:

1. Utilizando os códigos de Momentos Invariantes de HU ([código de exemplo](https://drive.matlab.com/sharing/03e51873-545c-4e56-8e5e-e52deb71bf79)). Existe outros códigos em Python, C, e outros
2. Utilize a base four shapes ([Base de Dados](https://www.dropbox.com/scl/fi/j23s8ck7brc1107b6jx3w/fourShapes_2.tar.gz?rlkey=hkvy84zqqukgmf833ae9i4x6a&dl=0));
3. Utilize o k-NN como classificador (neste momento só vamos usar o Classificador k-NN) - Pode usar pronto do Matlab ou do Scikit-learn.
4. Avalie com diferentes parâmetros (número de zonas horizontais e verticais) para os momentos invariantes de HU e apresente o melhor resultado.


In [19]:
!mkdir treino

mkdir: cannot create directory ‘treino’: File exists


In [20]:
!tar -xzf ./fourShapes.tar.gz -C ./treino/

In [21]:
!tar -xzf ./fourShapes_2.tar.gz -C ./treino/

In [22]:
!unzip ./formas_out_teste

Archive:  ./formas_out_teste.zip
replace formas_out/3_star/10_aug_1.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace formas_out/3_star/1002_aug_1.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [108]:
!unzip ./train_set.zip

Archive:  ./train_set.zip
   creating: train_set/
  inflating: train_set/.MATLABDriveTag  
   creating: train_set/0/
  inflating: train_set/0/.MATLABDriveTag  
  inflating: train_set/0/116150-6-3.bmp  
  inflating: train_set/0/116151-10-3.bmp  
  inflating: train_set/0/116151-10-4.bmp  
  inflating: train_set/0/116151-10-5.bmp  
  inflating: train_set/0/116151-13-3.bmp  
  inflating: train_set/0/116151-15-5.bmp  
  inflating: train_set/0/116151-17-4.bmp  
  inflating: train_set/0/116151-9-6.bmp  
  inflating: train_set/0/116152-19-4.bmp  
  inflating: train_set/0/116153-28-6.bmp  
  inflating: train_set/0/116154-3-6.bmp  
  inflating: train_set/0/116154-7-6.bmp  
  inflating: train_set/0/116154-9-4.bmp  
  inflating: train_set/0/116155-11-3.bmp  
  inflating: train_set/0/116156-1-4.bmp  
  inflating: train_set/0/116156-3-3.bmp  
  inflating: train_set/0/116158-4-5.bmp  
  inflating: train_set/0/116173-4-3.bmp  
  inflating: train_set/0/116174-6-3.bmp  
  inflating: train_set/0/116174-6

In [90]:
# Configurando o Ambiente
import glob

TRAINING_FILES = 600 # Quantidade de arquivos para treinamento

# Lista com o caminho das pastas
circlePath = ["./formas_out/1_circle/"]
squarePath = ["./formas_out/2_square/"]
starPath = ["./formas_out/3_star/"]
trianglePath = ["./formas_out/4_triangle/"]

In [91]:
# Função para listar os arquivos e separar os pirmeiros TRAINING_FILES como treino
def spread_test(folder_path):
  files = glob.glob(f"{folder_path}*.png")
  return (files[:TRAINING_FILES], files[TRAINING_FILES:])

# Função para listar os arquivos e separar os pirmeiros TRAINING_FILES como treino
def spread_training_test(folder_path: list[str]):
  trainign_files, test_files = spread_test(folder_path[-1])
  # for path in folder_path[:-1]:
  #   trainign_files.extend(glob.glob(f"{path}*.png"))
  return (trainign_files, test_files)

# Separando arquivo de treino e de teste
circleTraingFilesList, circleTestFilesList = spread_training_test(circlePath)
squareTraingFilesList, squareTestFilesList = spread_training_test(squarePath)
starTraingFilesList, starTestFilesList = spread_training_test(starPath)
triangleTraingFilesList, triangleTestFilesList = spread_training_test(trianglePath)

In [103]:
from PIL import Image
import numpy as np

# Função para abrir os arquivos como double e normalizar a imagem preto e branca
def open_files(file_list, threshold=0.5):
    np_array_list = []
    for file in file_list:
        img = Image.open(file).convert("L")
        img_array = np.array(img, dtype=np.float64) / 255.0
        # Binariza usando threshold fixo
        binary = (img_array > threshold).astype(np.float64)
        np_array_list.append(binary)
    return np.array(np_array_list)

# Abrindo arquivos de treino
X_train_circle = open_files(circleTraingFilesList)
X_train_square = open_files(squareTraingFilesList)
X_train_star = open_files(starTraingFilesList)
X_train_triangle = open_files(triangleTraingFilesList)

# Abrindo arquivos de teste
X_test_circle = open_files(circleTestFilesList)
X_test_square = open_files(squareTestFilesList)
X_test_star = open_files(starTestFilesList)
X_test_triangle = open_files(triangleTestFilesList)

In [104]:
# Juntar todos os treinos
X_train = np.concatenate([X_train_circle, X_train_square, X_train_star, X_train_triangle], axis=0)
X_test  = np.concatenate([X_test_circle, X_test_square, X_test_star, X_test_triangle], axis=0)

# Criar os rótulos (0=circle, 1=square, 2=star, 3=triangle)
y_train = np.concatenate([
    np.zeros(len(X_train_circle)),
    np.ones(len(X_train_square)),
    2*np.ones(len(X_train_star)),
    3*np.ones(len(X_train_triangle))
], axis=0)

y_test = np.concatenate([
    np.zeros(len(X_test_circle)),
    np.ones(len(X_test_square)),
    2*np.ones(len(X_test_star)),
    3*np.ones(len(X_test_triangle))
], axis=0)

In [105]:
from skimage.measure import moments_hu, moments

# Função para calcular momentos de Hu de todas as imagens
def extract_hu_features(images):
    features = []
    for img in images:
        m = moments(img)
        hu = moments_hu(m)
        # Normaliza os Momentos de HU
        hu = [-np.sign(h) * np.log10(abs(h)) if h != 0 else 0 for h in hu]
        features.append(hu)
    return np.array(features)

# Extrair features
X_train_features = extract_hu_features(X_train)
X_test_features  = extract_hu_features(X_test)

In [106]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Cria o classificador
knn = KNeighborsClassifier(n_neighbors=1)
# Resultados:
# Padrão (distância euclidiana), 1 vizinho me permitiu 0.764375 e 3 0.7775
# Manhattan, 1 vizinho me deu 76125 e 3 0.775

# Treina
knn.fit(X_train_features, y_train)

# Prediz
y_pred = knn.predict(X_test_features)

# Avalia
acc = accuracy_score(y_test, y_pred)
print("Acurácia do k-NN:", acc)

Acurácia do k-NN: 0.678125


In [101]:
# Funções de Hu Moments para calcular por zonas

# Função para dividir a imagem em zonas
def split_image(img, nx, ny):
    h, w = img.shape
    zones = []
    h_step = h // ny
    w_step = w // nx
    for i in range(ny):
        for j in range(nx):
            zone = img[i*h_step:(i+1)*h_step, j*w_step:(j+1)*w_step]
            zones.append(zone)
    return zones

# Função para calcular os momentos de hu em cada zona
def hu_features_zones(img, nx=2, ny=2):
    zones = split_image(img, nx, ny)
    features = []
    for z in zones:
        m = moments(z)
        hu = moments_hu(m)
        hu = [-np.sign(h) * np.log10(abs(h)) if h != 0 else 0 for h in hu]
        features.extend(hu)
    return np.array(features)

# Função para calcular os momentos de hu par auma lista de imagens
def extract_features_all(images, nx=2, ny=2):
    all_features = []
    for img in images:
        feat = hu_features_zones(img, nx, ny)
        all_features.append(feat)
    return np.array(all_features)

In [102]:
# Classificação com k-NN usando diferentes parâmetros
# (número de zonas horizontais e verticais) para os momentos invariantes de HU

best_acc = 0
best_params = (1,1)

for nx in range(1, 5):
    for ny in range(1, 5):
        # Extrai features
        X_train_f = extract_features_all(X_train, nx, ny)
        X_test_f  = extract_features_all(X_test, nx, ny)

        # Treina k-NN
        knn = KNeighborsClassifier(n_neighbors=3)
        knn.fit(X_train_f, y_train)
        y_pred = knn.predict(X_test_f)

        # Avalia
        acc = accuracy_score(y_test, y_pred)
        print(f"Acurácia {nx}x{ny} zonas: {acc:.4f}")

        # Guarda melhor resultado
        if acc > best_acc:
            best_acc = acc
            best_params = (nx, ny)

print(f"\nMelhor acurácia: {best_acc:.4f} com {best_params[0]}x{best_params[1]} zonas")

Acurácia 1x1 zonas: 0.6963
Acurácia 1x2 zonas: 0.8462
Acurácia 1x3 zonas: 0.9156
Acurácia 1x4 zonas: 0.9044
Acurácia 2x1 zonas: 0.8069
Acurácia 2x2 zonas: 0.8825
Acurácia 2x3 zonas: 0.9594
Acurácia 2x4 zonas: 0.9425
Acurácia 3x1 zonas: 0.9225
Acurácia 3x2 zonas: 0.9650
Acurácia 3x3 zonas: 0.8769
Acurácia 3x4 zonas: 0.9344
Acurácia 4x1 zonas: 0.8988
Acurácia 4x2 zonas: 0.9431
Acurácia 4x3 zonas: 0.9350
Acurácia 4x4 zonas: 0.8512

Melhor acurácia: 0.9650 com 3x2 zonas


In [133]:
TRAINING_FILES = 60 # Quantidade de arquivos para treinamento

# Lista com o caminho das pastas
zeroPath  = ["./train_set/0/"]
onePath   = ["./train_set/1/"]
twoPath   = ["./train_set/2/"]
threePath = ["./train_set/3/"]
fourPath  = ["./train_set/4/"]
fivePath  = ["./train_set/5/"]
sixPath   = ["./train_set/6/"]
sevenPath = ["./train_set/7/"]
eightPath = ["./train_set/8/"]
ninePath  = ["./train_set/9/"]

# Função para listar os arquivos e separar os pirmeiros TRAINING_FILES como treino
def spread_test(folder_path):
  files = glob.glob(f"{folder_path}*.bmp")
  return (files[:TRAINING_FILES], files[TRAINING_FILES:])

# Função para listar os arquivos e separar os pirmeiros TRAINING_FILES como treino
def spread_training_test(folder_path: list[str]):
  trainign_files, test_files = spread_test(folder_path[-1])
  # for path in folder_path[:-1]:
  #   trainign_files.extend(glob.glob(f"{path}*.png"))
  return (trainign_files, test_files)


# Separando arquivo de treino e de teste
zeroTraingFilesList, zeroTestFilesList = spread_training_test(zeroPath)
oneTraingFilesList, oneTestFilesList = spread_training_test(onePath)
twoTraingFilesList, twoTestFilesList = spread_training_test(twoPath)
threeTraingFilesList, threeTestFilesList = spread_training_test(threePath)
fourTraingFilesList, fourTestFilesList = spread_training_test(fourPath)
fiveTraingFilesList, fiveTestFilesList = spread_training_test(fivePath)
sixTraingFilesList, sixTestFilesList = spread_training_test(sixPath)
sevenTraingFilesList, sevenTestFilesList = spread_training_test(sevenPath)
eightTraingFilesList, eightTestFilesList = spread_training_test(eightPath)
nineTraingFilesList, nineTestFilesList = spread_training_test(ninePath)


def get_max_size(all_file_lists):
    max_w, max_h = 0, 0
    for file_list in all_file_lists:
        for file in file_list:
            img = Image.open(file).convert("L")
            w, h = img.size
            max_w = max(max_w, w)
            max_h = max(max_h, h)
    return max_w, max_h

all_train_lists = [
    zeroTraingFilesList, oneTraingFilesList, twoTraingFilesList,
    threeTraingFilesList, fourTraingFilesList, fiveTraingFilesList,
    sixTraingFilesList, sevenTraingFilesList, eightTraingFilesList,
    nineTraingFilesList
]

max_size = get_max_size(all_train_lists)


def open_files(file_list, max_size, threshold=0.5):
    max_w, max_h = max_size
    np_array_list = []
    for file in file_list:
        img = Image.open(file).convert("L")
        w, h = img.size

        # Criar imagem branca com tamanho maximo
        new_img = Image.new("L", (max_w, max_h), color=255)
        # Coloca a imagem no canto superior esquerdo
        x_offset = (max_w - w) // 2
        y_offset = (max_h - h) // 2
        new_img.paste(img, (x_offset, y_offset))

        img_array = np.array(new_img, dtype=np.float64) / 255.0
        binary = (img_array > threshold).astype(np.float64)
        np_array_list.append(binary)
    return np.array(np_array_list)


# Abrindo arquivos de treino
X_train_zero = open_files(zeroTraingFilesList, max_size)
X_train_one = open_files(oneTraingFilesList, max_size)
X_train_two = open_files(twoTraingFilesList, max_size)
X_train_three = open_files(threeTraingFilesList, max_size)
X_train_four = open_files(fourTraingFilesList, max_size)
X_train_five = open_files(fiveTraingFilesList, max_size)
X_train_six = open_files(sixTraingFilesList, max_size)
X_train_seven = open_files(sevenTraingFilesList, max_size)
X_train_eight = open_files(eightTraingFilesList, max_size)
X_train_nine = open_files(nineTraingFilesList, max_size)


# Abrindo arquivos de teste
X_test_zero = open_files(zeroTestFilesList, max_size)
X_test_one = open_files(oneTestFilesList, max_size)
X_test_two = open_files(twoTestFilesList, max_size)
X_test_three = open_files(threeTestFilesList, max_size)
X_test_four = open_files(fourTestFilesList, max_size)
X_test_five = open_files(fiveTestFilesList, max_size)
X_test_six = open_files(sixTestFilesList, max_size)
X_test_seven = open_files(sevenTestFilesList, max_size)
X_test_eight = open_files(eightTestFilesList, max_size)
X_test_nine = open_files(nineTestFilesList, max_size)


# Juntar todos os treinos
X_train = np.concatenate([
    X_train_zero, X_train_one, X_train_two, X_train_three,
    X_train_four, X_train_five, X_train_six, X_train_seven,
    X_train_eight, X_train_nine
], axis=0)
X_test  = np.concatenate([
    X_test_zero, X_test_one, X_test_two, X_test_three,
    X_test_four, X_test_five, X_test_six, X_test_seven,
    X_test_eight, X_test_nine
], axis=0)


# Criar os rotulos (0 a 9)
y_train = np.concatenate([
    np.full(len(X_train_zero), 0),
    np.full(len(X_train_one), 1),
    np.full(len(X_train_two), 2),
    np.full(len(X_train_three), 3),
    np.full(len(X_train_four), 4),
    np.full(len(X_train_five), 5),
    np.full(len(X_train_six), 6),
    np.full(len(X_train_seven), 7),
    np.full(len(X_train_eight), 8),
    np.full(len(X_train_nine), 9)
], axis=0)

y_test = np.concatenate([
    np.full(len(X_test_zero), 0),
    np.full(len(X_test_one), 1),
    np.full(len(X_test_two), 2),
    np.full(len(X_test_three), 3),
    np.full(len(X_test_four), 4),
    np.full(len(X_test_five), 5),
    np.full(len(X_test_six), 6),
    np.full(len(X_test_seven), 7),
    np.full(len(X_test_eight), 8),
    np.full(len(X_test_nine), 9)
], axis=0)


# Classificação com k-NN usando diferentes parâmetros
# (número de zonas horizontais e verticais) para os momentos invariantes de HU

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

best_acc = 0
best_params = (1,1)

for nx in range(1, 5):
    for ny in range(1, 5):
        # Extrai features
        X_train_f = extract_features_all(X_train, nx, ny)
        X_test_f  = extract_features_all(X_test, nx, ny)

        # Normaliza usando Z-score
        scaler = StandardScaler()
        X_train_f_scaled = scaler.fit_transform(X_train_f)
        X_test_f_scaled  = scaler.transform(X_test_f)

        # Treina k-NN
        knn = KNeighborsClassifier(n_neighbors=5)
        knn.fit(X_train_f_scaled, y_train)
        y_pred = knn.predict(X_test_f_scaled)

        # Avalia
        acc = accuracy_score(y_test, y_pred)
        print(f"Acurácia {nx}x{ny} zonas: {acc:.4f}")

        # Guarda melhor resultado
        if acc > best_acc:
            best_acc = acc
            best_params = (nx, ny)

print(f"\nMelhor acurácia: {best_acc:.4f} com {best_params[0]}x{best_params[1]} zonas")

Acurácia 1x1 zonas: 0.4650
Acurácia 1x2 zonas: 0.6875
Acurácia 1x3 zonas: 0.7075
Acurácia 1x4 zonas: 0.7625
Acurácia 2x1 zonas: 0.5525
Acurácia 2x2 zonas: 0.8225
Acurácia 2x3 zonas: 0.8725
Acurácia 2x4 zonas: 0.8925
Acurácia 3x1 zonas: 0.5725
Acurácia 3x2 zonas: 0.8050
Acurácia 3x3 zonas: 0.8650
Acurácia 3x4 zonas: 0.8475
Acurácia 4x1 zonas: 0.6075
Acurácia 4x2 zonas: 0.8825
Acurácia 4x3 zonas: 0.7850
Acurácia 4x4 zonas: 0.8725

Melhor acurácia: 0.8925 com 2x4 zonas
